# Helper Functions

| Date | Description |
| --- | --- |
| 2022-08-05 | Initial Creation |

## Load Libraries

In [ ]:
import numpy as np
import pandas as pd

## Functions

### world_bank_file_to_df()

In [1]:
def world_bank_file_to_df(filepath):
    # read the file and then drop NA to remove extra rows
    # only headers and actual rows remain (blank cells are marked by '..' so they won't be NULL)
    outdf = pd.read_csv(filepath).dropna()

    # Unpivot the 'year' columns so that they would just have one column
    # Set the indicator under the column "value" for now
    col_list = list(outdf.columns)
    # First 4 columns is always Country, Country Code, Series Code, Series Name
    idcols = col_list[:4]
    varcols = col_list[4:]

    outdf = pd.melt(outdf, id_vars=idcols, value_vars=varcols, var_name='Year', value_name='value')

    outdf['value'] = outdf['value'].replace(r'\.\.',np.nan,regex=True)

    # create a data frame to have the series code and its name
    desc_df = outdf.copy()
    desc_df = desc_df[['Series Code', 'Series Name']].drop_duplicates()

    # now we pivot the table so that the 'series'/indicator be shown as columns 
    outdf = pd.pivot(outdf, index=['Country Name', 'Country Code', 'Year'], columns=['Series Code'], values='value')
    outdf = outdf.reset_index()
    outdf.columns.name = None

    outdf = outdf.drop('Country Code',axis=1)
    outdf.rename(columns={'Country Name':'Country'},inplace=True)

    outdf['Year']=outdf['Year'].str.replace('\s\[.*\]', '', regex=True).astype(int)

    for col in list(outdf.columns)[2:]:
        outdf[col] = outdf[col].astype(float)

    return desc_df, outdf





In [ ]:
def visualize_nulls(df):
    plt.figure(figsize=(12,10))
    ax = sns.heatmap(df[list(df.columns)[2:]].isnull(),cbar=False)
    ax.set_xlabel('Indicators')
    plt.show()

    print('Percentage of Null values in Dataset:')
    display(df[list(df.columns)[2:]].isnull().sum()/df.shape[0] *100)

    yr_list = sorted(list(set(df['Year'].tolist())))
    
    for yr in yr_list:
        tempdf = df[df['Year'] == yr]
        print('Year =', yr)
        display(tempdf[list(tempdf.columns)[2:]].isnull().sum()/tempdf.shape[0]*100)

    

In [ ]:
def get_yoy_change_df(df, colname, changecolname):
    tempdf = df[['Country', 'Year', colname]].copy()
    yr_list = sorted(list(set(tempdf['Year'].tolist())))
    outdf = pd.pivot(tempdf, index='Country', columns='Year', values=colname)
    for yr in yr_list[1:]:
        outdf[str(yr)+'_chg'] = (outdf[yr]-outdf[yr-1])/outdf[yr-1]*100
        outdf = outdf.drop(yr-1,axis='columns')
    outdf = outdf.drop(yr,axis='columns')
    
    outdf = outdf.reset_index()
    outdf.columns.name = None
    outdf = pd.melt(outdf, id_vars='Country', value_vars=list(outdf.columns)[1:], var_name='Year', value_name=changecolname)
    outdf['Year']=outdf['Year'].str.replace('_.*', '', regex=True).astype(int)

    return outdf